In [1]:
import os
import torch

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


emotions_dict = {
    '0': 'neutral',
    '1': 'calm',
    '2': 'happy',
    '3': 'sad',
    '4': 'angry',
    '5': 'fearful',
    '6': 'disgust',
    '7': 'surprised'
}

fold_path = 'scaling-spectrogram-mfcc dataset'
sub_name = 'LIGHT-SERNET melspectrogram+mfcc-deltas res-conn'
data_name = 'RAVDESS'

minibatch = 32
num_epochs = 500
EARLY_CNT = 99999

learning_rate = 1e-5

In [2]:
from cv2 import transform
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
from einops import rearrange
import torch.nn.functional as F
from torchinfo import summary
from tqdm.auto import tqdm
import math
from torchaudio.functional import compute_deltas
import random

In [3]:
def set_seed(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
    if torch.backends.cudnn.is_available:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    print(f'# SEEDING DONE : seed is {seed_value} ')

set_seed(2022)

# SEEDING DONE : seed is 2022 


In [4]:
def get_padding(kernel_size: int, stride: int, dilation: int) -> int:
    if stride > 1 and dilation > 1:
        raise ValueError('Only stride OR dilation may be greater than 1')
    return (kernel_size // 2) * dilation

In [5]:
class LIGHT_SERNET(nn.Module):
    def __init__(self):
        super(LIGHT_SERNET, self).__init__()

        self.cnn1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=[3,3], padding=[1,0]),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d([2,2], padding=[1,1])
        )
        self.cnn2 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=[9,1], padding=[4,0]),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d([2,2], padding=[1,1])
        )
        self.cnn3 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=[1,11]),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d([2,2], padding=[1,1])
        )
        
        self.LFLB = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=[3,3], padding=[1,1]),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d([2,2], padding=[1,1]),
            
            nn.Conv2d(64, 96, kernel_size=[3,3], padding=[1,1]),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d([2,2], padding=[1,1]),
            
            nn.Conv2d(96, 128, kernel_size=[3,3], padding=[1,1]),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d([2,1], padding=[1,0]),
            
            nn.Conv2d(128, 160, kernel_size=[3,3], padding=[1,1]),
            nn.BatchNorm2d(160),
            nn.ReLU(),
            nn.MaxPool2d([2,1], padding=[1,0]),
            
            nn.Conv2d(160, 320, kernel_size=[1,1]),
            nn.BatchNorm2d(320),
            nn.ReLU(),
            
            nn.Conv2d(320, 640, kernel_size=[1,1]),
            nn.BatchNorm2d(640),
            nn.ReLU()
        )

    def forward(self, x):
        
        embd1 = self.cnn1(x)
        embd2 = self.cnn2(x)
        embd3 = self.cnn3(x)
        concat_embd = torch.cat([embd1, embd2, embd3], dim=3)
        
        lflb_embd = self.LFLB(concat_embd)
        out = F.adaptive_avg_pool2d(lflb_embd, (1,1))
        out = nn.Dropout(0.3)(out)
        out = torch.flatten(out, start_dim=1)
        
        return out
    

class Model(nn.Module):
    """Some Information about Model"""
    def __init__(self, num_emotions):
        super(Model, self).__init__()

        self.mel = LIGHT_SERNET()
        self.mfcc = LIGHT_SERNET()
        
        self.fc = nn.Linear(640+7360*2, num_emotions)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        mel = x[:,0,:,:]
        mel = mel.unsqueeze(1)
        
        mel_delta = compute_deltas(mel, win_length=7)
        mel_delta = torch.add(mel, mel_delta)
        
        mel_delta2 = compute_deltas(mel_delta, win_length=7)
        mel_delta2 = torch.add(mel, mel_delta2)
        
        mel_embd = torch.cat([mel, mel_delta, mel_delta2], dim=1)
        
        mfcc = x[:,1,:,:]
        mfcc = mfcc.unsqueeze(1)
        
        mfcc_delta = compute_deltas(mfcc, win_length=7)
        mfcc_delta = torch.add(mfcc, mfcc_delta)
        
        mfcc_delta2 = compute_deltas(mfcc_delta, win_length=7)
        mfcc_delta2 = torch.add(mfcc, mfcc_delta2)
        
        mfcc_embd = torch.cat([mfcc, mfcc_delta, mfcc_delta2], dim=1)

        mel_embd = self.mel(mel_embd)
        mfcc_embd = self.mfcc(mfcc_embd)
        
        mel_origin = torch.flatten(mel, start_dim=1)
        mfcc_origin = torch.flatten(mfcc, start_dim=1)
        
        output = torch.maximum(mel_embd, mfcc_embd)
        
        output = torch.cat([output, mel_origin, mfcc_origin], dim=1)
        
        output_logits = self.fc(output)
        output_softmax = self.softmax(output_logits)

        return output_logits, output_softmax
    
model = Model(8)
print(summary(model, (1140, 2, 40, 184)))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [1140, 8]                 --
├─LIGHT_SERNET: 1-1                      [1140, 640]               --
│    └─Sequential: 2-1                   [1140, 32, 21, 92]        --
│    │    └─Conv2d: 3-1                  [1140, 32, 40, 182]       896
│    │    └─BatchNorm2d: 3-2             [1140, 32, 40, 182]       64
│    │    └─ReLU: 3-3                    [1140, 32, 40, 182]       --
│    │    └─MaxPool2d: 3-4               [1140, 32, 21, 92]        --
│    └─Sequential: 2-2                   [1140, 32, 21, 93]        --
│    │    └─Conv2d: 3-5                  [1140, 32, 40, 184]       896
│    │    └─BatchNorm2d: 3-6             [1140, 32, 40, 184]       64
│    │    └─ReLU: 3-7                    [1140, 32, 40, 184]       --
│    │    └─MaxPool2d: 3-8               [1140, 32, 21, 93]        --
│    └─Sequential: 2-3                   [1140, 32, 21, 88]        --
│    │    └─C

In [6]:
def make_train_step(model, criterion, optimizer):

    # define the training step of the training phase
    def train_step(X, Y):

        # forward pass
        output_logits, output_softmax = model(X)
        predictions = torch.argmax(output_softmax, dim=1)
        accuracy = torch.sum(Y == predictions)/float(len(Y))

        # compute loss on logits because nn.CrossEntropyLoss implements log softmax
        loss = criterion(output_logits, Y)

        # compute gradients for the optimizer to use
        loss.backward()

        # update network parameters based on gradient stored (by calling loss.backward())
        optimizer.step()

        # zero out gradients for next pass
        # pytorch accumulates gradients from backwards passes (convenient for RNNs)
        optimizer.zero_grad()

        return loss.item(), accuracy*100
    return train_step

In [7]:
def make_validate_fnc(model, criterion):
    def validate(X, Y):

        # don't want to update any network parameters on validation passes: don't need gradient
        # wrap in torch.no_grad to save memory and compute in validation phase:
        with torch.no_grad():

            # set model to validation phase i.e. turn off dropout and batchnorm layers
            model.eval()
            
            # get the model's predictions on the validation set
            output_logits, output_softmax = model(X)
            predictions = torch.argmax(output_softmax, dim=1)

            # calculate the mean accuracy over the entire validation set
            accuracy = torch.sum(Y == predictions)/float(len(Y))

            # compute error from logits (nn.crossentropy implements softmax)
            loss = criterion(output_logits, Y)

        return loss.item(), accuracy*100, predictions
    return validate

In [8]:
def make_save_checkpoint():
    def save_checkpoint(optimizer, model, epoch, filename):
        checkpoint_dict = {
            'optimizer': optimizer.state_dict(),
            'model': model.state_dict(),
            'epoch': epoch
        }
        torch.save(checkpoint_dict, filename)
    return save_checkpoint


def load_checkpoint(optimizer, model, filename):
    checkpoint_dict = torch.load(filename)
    epoch = checkpoint_dict['epoch']
    model.load_state_dict(checkpoint_dict['model'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint_dict['optimizer'])
    return epoch

In [9]:
def criterion(predictions, targets):
    return nn.CrossEntropyLoss()(input=predictions, target=targets)

In [10]:
from einops import rearrange

def train(model, num_epochs, X_train, Y_train, X_valid, Y_valid, pkl_name, fold, optimizer):
    
    # instantiate lists to hold scalar performance metrics to plot later
    train_losses = []
    valid_losses = []

    # get training set size to calculate # iterations and minibatch indices
    train_size = X_train.shape[0]

    # instantiate the checkpoint save function
    save_checkpoint = make_save_checkpoint()

    # instantiate the training step function
    train_step = make_train_step(model, criterion, optimizer=optimizer)
    # instantiate the validation loop function
    validate = make_validate_fnc(model, criterion)

    cur_valid_loss = 0x3f3f3f
    early_stop_cnt = 0
    
    for epoch in tqdm(range(num_epochs), desc=f'fold-{fold}'):

        if early_stop_cnt >= EARLY_CNT:
            break
        
        # set model to train phase
        model.train()

        # shuffle entire training set in each epoch to randomize minibatch order
        train_indices = np.random.permutation(train_size)

        # shuffle the training set for each epoch:
        X_train = X_train[train_indices, :, :]
        Y_train = Y_train[train_indices]

        # instantiate scalar values to keep track of progress after each epoch so we can stop training when appropriate
        epoch_acc = 0
        epoch_loss = 0
        num_iterations = int(train_size / minibatch)

        # create a loop for each minibatch of 32 samples:
        for i in range(num_iterations):

            # we have to track and update minibatch position for the current minibatch
            # if we take a random batch position from a set, we almost certainly will skip some of the data in that set
            # track minibatch position based on iteration number:
            batch_start = i * minibatch
            # ensure we don't go out of the bounds of our training set:
            batch_end = min(batch_start + minibatch, train_size)
            # ensure we don't have an index error
            actual_batch_size = batch_end-batch_start

            # get training minibatch with all channnels and 2D feature dims
            X = X_train[batch_start:batch_end, :, :]
            # get training minibatch labels
            Y = Y_train[batch_start:batch_end]
            
            # instantiate training tensors
            X_tensor = torch.tensor(X, device=device).float()
            Y_tensor = torch.tensor(Y, dtype=torch.long, device=device)
            
            # Pass input tensors thru 1 training step (fwd+backwards pass)
            loss, acc = train_step(X_tensor, Y_tensor)

            # aggregate batch accuracy to measure progress of entire epoch
            epoch_acc += acc * actual_batch_size / train_size
            epoch_loss += loss * actual_batch_size / train_size

            # keep track of the iteration to see if the model's too slow
            print('\r'+f'Epoch {epoch}: iteration {i}/{num_iterations}', end='')

        X_valid_tensor = torch.tensor(X_valid, device=device).float()
        Y_valid_tensor = torch.tensor(Y_valid, dtype=torch.long, device=device)
        
        # calculate validation metrics to keep track of progress; don't need predictions now
        valid_loss, valid_acc, _ = validate(X_valid_tensor, Y_valid_tensor)
        
        # accumulate scalar performance metrics at each epoch to track and plot later
        train_losses.append(epoch_loss)
        valid_losses.append(valid_loss)
        
        if cur_valid_loss > valid_loss:
            save_checkpoint(optimizer, model, epoch, pkl_name)
            cur_valid_loss = valid_loss
        else:
            early_stop_cnt += 1

        # keep track of each epoch's progress
        print(f'\nEpoch {epoch} --- loss:{epoch_loss:.3f}, Epoch accuracy:{epoch_acc:.2f}%, Validation loss:{valid_loss:.3f}, Validation accuracy:{valid_acc:.2f}%')

        
    print(f"\n\n[*] done !")
    print(f'[*] Best training loss - {min(train_losses)}')
    print(f'[*] Best validation loss - {min(valid_losses)}')
    
    return train_losses, valid_losses

In [11]:
def get_fold_dataset(npy_name):
    with open(npy_name, 'rb') as f:
        X_train = np.load(f)
        y_train = np.load(f)
        X_test = np.load(f)
        y_test = np.load(f)
    
    return X_train, y_train, X_test, y_test

In [12]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

train_losses, valid_losses = [], []

print(f'[*] model training - {sub_name}')

for fold in range(5):
    cur_fold_path = os.path.join(fold_path, 'fold'+str(fold)+'.npy')
    X_train, y_train, X_test, y_test = get_fold_dataset(cur_fold_path)
    
    pkl_name = f'./model/{data_name}-{sub_name}-{fold}.pkl'
    model = Model(num_emotions=len(emotions_dict)).to('cuda')
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=10e-6, amsgrad=True)
    
    train_losses, valid_losses = train(model, num_epochs, X_train, y_train, X_test, y_test, pkl_name, fold, optimizer)
    
    plt.title('Loss Curve')
    plt.ylabel('Loss', fontsize=16)
    plt.xlabel('Epoch', fontsize=16)
    plt.plot(train_losses[:],'b')
    plt.plot(valid_losses[:],'r')
    plt.legend(['Training loss','Validation loss'])
    plt.show()

[*] model training - LIGHT-SERNET melspectrogram+mfcc-deltas res-conn


A Jupyter Widget

Epoch 0: iteration 34/35
Epoch 0 --- loss:27.115, Epoch accuracy:12.81%, Validation loss:13.169, Validation accuracy:14.33%
Epoch 1: iteration 34/35
Epoch 1 --- loss:9.314, Epoch accuracy:20.00%, Validation loss:9.579, Validation accuracy:13.00%
Epoch 2: iteration 34/35
Epoch 2 --- loss:7.402, Epoch accuracy:22.72%, Validation loss:8.728, Validation accuracy:17.00%
Epoch 3: iteration 34/35
Epoch 3 --- loss:6.500, Epoch accuracy:24.47%, Validation loss:7.820, Validation accuracy:20.33%
Epoch 4: iteration 34/35
Epoch 4 --- loss:5.869, Epoch accuracy:26.75%, Validation loss:7.155, Validation accuracy:18.00%
Epoch 5: iteration 34/35
Epoch 5 --- loss:5.093, Epoch accuracy:30.09%, Validation loss:6.820, Validation accuracy:22.67%
Epoch 6: iteration 34/35
Epoch 6 --- loss:4.427, Epoch accuracy:35.09%, Validation loss:6.534, Validation accuracy:21.33%
Epoch 7: iteration 34/35
Epoch 7 --- loss:4.344, Epoch accuracy:36.49%, Validation loss:7.226, Validation accuracy:25.67%
Epoch 8: iteration 34/

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

device = 'cuda'
model = Model(num_emotions=len(emotions_dict)).to('cuda')

test_accs = []

for fold in range(5):
    pkl_name = f'./model/{data_name}-{sub_name}-{fold}.pkl'
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0., amsgrad=True)
    
    load_checkpoint(optimizer, model, pkl_name)
    
    validate = make_validate_fnc(model, criterion)
    
    cur_fold_path = os.path.join(fold_path, 'fold'+str(fold)+'.npy')
    X_train, y_train, X_test, y_test = get_fold_dataset(cur_fold_path)
    
    X_test_tensor = torch.tensor(X_test,device=device).float()
    # X_test_tensor = X_test_tensor.permute(0, 3, 1, 2)
    y_test_tensor = torch.tensor(y_test,dtype=torch.long,device=device)
    
    test_loss, test_acc, predicted_emotions = validate(X_test_tensor, y_test_tensor)
    print(f'[*] {fold} Test accuracy is {test_acc:.2f}%')
    test_accs.append(test_acc)
    
    predicted_emotions = predicted_emotions.cpu().numpy()
    # use labels from test set
    emotions_groundtruth = y_test_tensor.cpu().numpy()
    
    # build confusion matrix and normalized confusion matrix
    conf_matrix = confusion_matrix(emotions_groundtruth, predicted_emotions)
    conf_matrix_norm = confusion_matrix(emotions_groundtruth, predicted_emotions,normalize='true')

    # set labels for matrix axes from emotions
    emotion_names = [emotion for emotion in emotions_dict.values()]

    # make a confusion matrix with labels using a DataFrame
    confmatrix_df = pd.DataFrame(conf_matrix, index=emotion_names, columns=emotion_names)
    confmatrix_df_norm = pd.DataFrame(conf_matrix_norm, index=emotion_names, columns=emotion_names)

    # plot confusion matrices
    plt.figure(figsize=(16,6))
    sn.set(font_scale=1.8) # emotion label and title size
    plt.subplot(1,2,1)
    plt.title('Confusion Matrix')
    sn.heatmap(confmatrix_df, annot=True, annot_kws={"size": 18}) #annot_kws is value font
    plt.subplot(1,2,2)
    plt.title('Normalized Confusion Matrix')
    sn.heatmap(confmatrix_df_norm, annot=True, annot_kws={"size": 13}) #annot_kws is value font

    plt.show()

In [ ]:
def Average(lst):
    return sum(lst) / len(lst)

print('[*] K-Fold training done !')
print(f'[*] Best fold acc : {max(test_accs):.2f}%')
print(f'[*] Worst fold acc : {min(test_accs):.2f}%')
print(f'[*] Average fold acc : {Average(test_accs):.2f}%')

In [ ]:
from sklearn.metrics import classification_report

device = torch.device('cpu')
model = model.to(device)

test_accs = []

for fold in range(5):
    pkl_name = f'./model/{data_name}-{sub_name}-{fold}.pkl'
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0., amsgrad=True)
    
    load_checkpoint(optimizer, model, pkl_name)
    
    validate = make_validate_fnc(model, criterion)
    
    cur_fold_path = os.path.join(fold_path, 'fold'+str(fold)+'.npy')
    X_train, y_train, X_test, y_test = get_fold_dataset(cur_fold_path)
    
    X_test_tensor = torch.tensor(X_test).float()
    # X_test_tensor = X_test_tensor.permute(0, 3, 1, 2)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    
    test_loss, test_acc, predicted_emotions = validate(X_test_tensor, y_test_tensor)
    print(f'[*] {fold} Test accuracy is {test_acc:.2f}%')
    test_accs.append(test_acc)
    
    predicted_emotions = predicted_emotions.cpu().numpy()
    # use labels from test set
    emotions_groundtruth = y_test_tensor

    print(classification_report(emotions_groundtruth, predicted_emotions))